# High-level Keras R (TF) CNN Example

In [1]:
# SETUP
#
# Install keras R
# install.packages('keras', repos = "https://cloud.r-project.org")
# 
# Update reticulate from cran (it defaults to mran which has an outdated version)
# install.packages("reticulate", repos = "https://cloud.r-project.org")

In [2]:
library(keras)
use_python('/anaconda/envs/py35')

# Import util functions
source("./common/utils.R")

# Import hyper-parameters
params <- load_params("cnn")

Loading required package: rjson


In [3]:
# Performance Improvement
# 1. Make sure channels-first (not last)
backend()$set_image_data_format('channels_first')

In [4]:
# reticulate::py_config()

In [5]:
cat("OS:", Sys.info()["sysname"], "\n")
cat(R.version$version.string, "\n")
cat("Keras:", paste0(packageVersion("keras")), "\n")
cat("Tensorflow:", paste0(packageVersion("tensorflow")), "\n")
cat("Keras using", backend()$backend(), "\n")
cat("Keras channel ordering is", backend()$image_data_format(), "\n") 
cat("GPU: ", get_gpu_name(), "\n")
cat(get_cuda_version(), "\n")
cat(get_cudnn_version(), "\n")

OS: Linux 
R version 3.4.1 (2017-06-30) 
Keras: 2.1.5 
Tensorflow: 1.5 
Keras using tensorflow 
Keras channel ordering is channels_first 
GPU:  Tesla P100-PCIE-16GB 
CUDA Version 8.0.61 
CuDNN Version 6.0.21 


In [6]:
create_symbol <- function(n_classes = params$N_CLASSES){
    
    # initialize a sequential model
    model <- keras_model_sequential() %>%
    
    # 2D convolutional layers being fed 32x32 pixel images   
    layer_conv_2d(filters = 50, kernel_size = c(3, 3), padding = "same", activation = "relu", input_shape = c(3, 32, 32)) %>%
    layer_conv_2d(filters = 50, kernel_size = c(3, 3), padding = "same", activation = "relu") %>%
    
    # max pooling
    layer_max_pooling_2d(pool_size = c(2, 2), strides = c(2, 2)) %>%
    layer_dropout(0.25) %>%

    # 2D convolutional layers 
    layer_conv_2d(filters = 100, kernel_size = c(3, 3), padding = "same", activation = "relu") %>%
    layer_conv_2d(filters = 100, kernel_size = c(3, 3), padding = "same", activation = "relu") %>%
    
    # max pooling
    layer_max_pooling_2d(pool_size = c(2, 2), strides = c(2, 2)) %>%
    layer_dropout(0.25) %>%

    # flatten into feature vector
    layer_flatten() %>%
    layer_dense(512, activation = "relu") %>%
    layer_dropout(0.5) %>%
    layer_dense(n_classes, activation = "softmax")  
    
    return(model)
}

In [7]:
init_model <- function(m, lr=params$LR, momentum=params$MOMENTUM){
    m %>% compile(
      loss = "categorical_crossentropy",
      optimizer = optimizer_sgd(lr, momentum),
      metrics = "accuracy"
    )
    return(m)
}

In [8]:
# Data Preparation 
cifar <- cifar_for_library(one_hot = TRUE, col_major = FALSE)
x_train <- cifar$x_train
y_train <- cifar$y_train
x_test <- cifar$x_test
y_test <- cifar$y_test

rm(cifar)

[1] "Data does not exist. Downloading https://ikpublictutorial.blob.core.windows.net/deeplearningframeworks/cifar-10-binary.tar.gz "
[1] "Extracting files ..."


In [9]:
cat('x_train shape:', dim(x_train), '\n')
cat('x_test shape:', dim(x_test), '\n')
cat('y_train shape:', dim(y_train), '\n')
cat('y_test shape:', dim(y_test), '\n')

x_train shape: 50000 3 32 32 
x_test shape: 10000 3 32 32 
y_train shape: 50000 10 
y_test shape: 10000 10 


In [10]:
# Load symbol
sym = create_symbol()

In [11]:
# Initialise model
model = init_model(sym)

In [12]:
summary(model)

________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
conv2d_1 (Conv2D)                   (None, 50, 32, 32)              1400        
________________________________________________________________________________
conv2d_2 (Conv2D)                   (None, 50, 32, 32)              22550       
________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)      (None, 50, 16, 16)              0           
________________________________________________________________________________
dropout_1 (Dropout)                 (None, 50, 16, 16)              0           
________________________________________________________________________________
conv2d_3 (Conv2D)                   (None, 100, 16, 16)             45100       
________________________________________________________________________________
conv2d_4 (Conv2D)           

In [13]:
# Main training loop
system.time(
    model %>% fit(
        x_train, y_train,
        batch_size = params$BATCHSIZE,
        epochs = params$EPOCHS)
)

   user  system elapsed 
 69.565  14.586  71.634 

In [14]:
# Main evaluation loop
y_guess <- model %>% predict_classes(x_test, batch_size = params$BATCHSIZE)
y_truth <- apply(y_test, 1, function(x) which.max(x)-1)

In [15]:
print(paste0("Accuracy: ", sum(y_guess == y_truth)/length(y_guess)))

[1] "Accuracy: 0.7685"
